In [1]:
isOpenset = True
existSavedFile = True

isLocalServer = True
sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID_model_rebuild/'
config_path = './configs/config.json'

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os

def checkLocalServer(isLocalServer, sys_path_to_be_added=None):
    print(f'isLocalServer is {isLocalServer}')
    if isLocalServer is True:
        if sys_path_to_be_added not in sys.path:
            sys.path.insert(0, sys_path_to_be_added)
            os.chdir(sys_path_to_be_added)

checkLocalServer(isLocalServer, sys_path_to_be_added)

from utils import Config_Manager
cfgm = Config_Manager(config_path)
cfg = cfgm.get_cfg()
!pwd
!nvidia-smi

isLocalServer is True
/notebook/personal/ksuchoi216/FaceID_model_rebuild
Tue Oct 18 13:48:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:04:00.0 Off |                    0 |
| N/A   33C    P0    33W / 250W |   2392MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P1

In [2]:
from data_preprocessing import FolderDataset, DatasetFromNumpy
from data_preprocessing import buildDataLoaders, saveDataloaders

cfg["FolderDataset"]["folder_for_raw"] = cfg["folder_for_raw"]
folderDataset = FolderDataset(cfg["FolderDataset"])

folderDataset.setFilePath(
    './data',
    'extracted_org',
    'face',
    '.npy'
)

if not existSavedFile:
    print("loading numpy datasets...")
    emb, lb = folderDataset.createNumpyData()
    folderDataset.saveToNumpy(emb, lb)
emb, lb = folderDataset.loadFromNumpy()
print("creating a dataset from numpy datasets")



device is cuda:0
image size: 140
Loading faces from ./raw/photos_from_video
0 jhoh
1 jhongyoo
2 jhyoo
3 kschoi
path_emb: ./data/extracted_org/face_emb.npy
path_lb: ./data/extracted_org/face_lb.npy
loaded emb:(1157, 512) lb:(1157,)
creating a dataset from numpy datasets


In [10]:
from data_preprocessing import filterNumpy, splitNumpy
from data_preprocessing import putTogetherData, convertLabels
from data_preprocessing import buildMultipleDataLoaders

if isOpenset:
    kc = [0, 1, 2] #"known known class + known unknown class"
    ukc = [3] #"unknown known class"
    
    emb_kc, lb_kc = filterNumpy(
        data = emb,
        labels = lb,
        selected_labels = kc
    )
    
    emb_ukc, lb_ukc = filterNumpy(
        data = emb,
        labels = lb,
        selected_labels = ukc
    )
    
    lb_kc = convertLabels(
        labels=lb_kc,
        selected_labels=[0, 1],
        marked_label = 1,
        unmarked_label = 0
        )
    lb_ukc = convertLabels(
        labels=lb_ukc,
        selected_labels=[3],
        marked_label = 0,
        unmarked_label = 2,
        )
    
    
    kc_ratios = {'train': 0.7, 'val': 0.2, 'test': 0.1}
    data_kc = splitNumpy(emb_kc, lb_kc, ratios=kc_ratios)
    
    ukc_ratios = {'val': 0.6, 'test': 0.4}
    data_ukc = splitNumpy(emb_ukc, lb_ukc, ratios=ukc_ratios)
    
    total_data = putTogetherData(data_kc, data_ukc)
    
    dataloaders = buildMultipleDataLoaders(total_data, batch_size=4)
    saveDataloaders(source_folder='data',
                    save_folder='dataloader_osr',
                    dataloaders=dataloaders)
    
else:
    dataloaders = buildDataLoaders(emb, lb,
                                   batch_size=4,
                                   ratio_train=0.7,
                                   ratio_val=0.2)
    saveDataloaders(source_folder='data',
                    save_folder='dataloader_org',
                    dataloaders=dataloaders)



[FilterNumpy]
converted from (1157, 512) to (895, 512)
converted from (895,) to (895,)


[FilterNumpy]
converted from (1157, 512) to (262, 512)
converted from (262,) to (262,)
num_m_lb = 595, num_unm_lb = 300 set: {0, 1}
num_m_lb = 262, num_unm_lb = 0 set: {0}


[splitNumpy]
[train exist] Ratios: remain 0.3 test 0.33
dict_keys(['train', 'val', 'test'])


[splitNumpy]
[no train exist] There is no train ratio in ratios
dict_keys(['val', 'test'])


[putTogether]
dict_keys(['train', 'val', 'test'])
(626, 512) (626,)
(337, 512) (337,)
(194, 512) (194,)


[Build dataloaders]
[train] labels: {0, 1}
dataset initial emb:(626, 512) lb:(626,)
[val] labels: {0, 1}
dataset initial emb:(337, 512) lb:(337,)
[test] labels: {0, 1}
dataset initial emb:(194, 512) lb:(194,)
saved in ./data/dataloader_osr/dataloader_train.pt
saved in ./data/dataloader_osr/dataloader_val.pt
saved in ./data/dataloader_osr/dataloader_test.pt
